<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 19 16:47:24 2022

@author: Samer Chaaraoui

from paper: Schroedter-Homscheidt, Marion und Kosmale, Miriam und Jung, Sandra und Kleissl, Jan (2018)
Classifying ground-measured 1 minute temporal variability within hourly intervals for direct normal irradiances. Meteorologische Zeitschrift, 27 (2), Seiten 161-179. Borntraeger Science Publishers.
doi: 10.1127/metz/2018/0875. ISSN 0941-2948.
"""

# Import der benötigten Module
import pandas as pd
import pvlib
from math import sqrt
import numpy as np
from datetime import timedelta

# tz = timezone
latitude, longitude, tz, altitude, name = 38.642, -121.148, 'America/Los_Angeles', 100, 'Folsom'
# tus = Tucson - aus Tutorial keine Bedeutung
tus = pvlib.location.Location(latitude, longitude, tz, altitude, name)

# Klasse für den Klassifokator
class Homscheidt_Functions:
    def __init__(self,
                 rad_df: pd.DataFrame):
        """
        Parameters
        ----------------
        rad_df: dataframe of the past radiation data observations

        """
        self.rad_df = rad_df
        self.cs = tus.get_clearsky(rad_df.index)

    def sigma_skartveit_GHI(self):
        # Index nach Skarveit
        # berechnen des Variabiltätsindex sigma_skarveit_GHI mit Hilfe der Globalstrahlung
        """
        Eq. (3.1)
        paper uses DNI values. Here GHI
        can reach values above 1
        Skartveit A., J.A. Olseth, M.E. Tuft, 1998: An hourly
        diffuse fraction model with correction for variability
        and surface albedo. – Sol. Energy 63, 173–183, DOI:
        10.1016/S0038-092X(98)00067-X.

        Returns
        -------
        sigma_skartveit

        """
        # kc = Clear-Sky-Index | kc_t_m1 = Clear-Sky-Index of previus hour | kc_t_p1 = Clear-Sky-Index following hour
        kc = []
        kc_t_m1 = []
        kc_t_p1 = []
        # füllen von kc | kc_t_m1 | kc_t_p1
        for i in range(1,len(self.rad_df['ghi'])-1):
            kc.append(self.rad_df['ghi'].iloc[i]/self.cs['ghi'].iloc[i])
            kc_t_m1.append(self.rad_df['ghi'].iloc[i-1]/self.cs['ghi'].iloc[i-1])
            kc_t_p1.append(self.rad_df['ghi'].iloc[i+1]/self.cs['ghi'].iloc[i+1])

        # Berechnung der arthmetischen Mittelwerte
        kc = np.array(kc).mean()
        kc_t_m1 = np.array(kc_t_m1).mean()
        kc_t_p1 = np.array(kc_t_p1).mean()

        # Umsetzung der Gleichung 3.1 für die GHI
        return sqrt((((kc-kc_t_m1)**2)+((kc-kc_t_p1)**2))/2)

    def sigma_skartveit_DNI(self):
        # Index nach Skarveit
        # berechnen des Variabiltätsindex sigma_skarveit_DNI mit Hilfe der Clear-Sky-Direktstrahlung
        """
        Eq. (3.1)
        paper uses DNI values.
        can reach values above 1
        Skartveit A., J.A. Olseth, M.E. Tuft, 1998: An hourly
        diffuse fraction model with correction for variability
        and surface albedo. – Sol. Energy 63, 173–183, DOI:
        10.1016/S0038-092X(98)00067-X.

        Returns
        -------
        sigma_skartveit

        """
        # kc = Clear-Sky-Index | kc_t_m1 = Clear-Sky-Index of previus hour | kc_t_p1 = Clear-Sky-Index following hour
        kc = []
        kc_t_m1 = []
        kc_t_p1 = []

        # füllen von kc | kc_t_m1 | kc_t_p1
        for i in range(1,len(self.rad_df['dni'])-1):
            kc.append(self.rad_df['dni'].iloc[i]/self.cs['dni'].iloc[i])
            kc_t_m1.append(self.rad_df['dni'].iloc[i-1]/self.cs['dni'].iloc[i-1])
            kc_t_p1.append(self.rad_df['dni'].iloc[i+1]/self.cs['dni'].iloc[i+1])


        kc = np.array(kc).mean()
        kc_t_m1 = np.array(kc_t_m1).mean()
        kc_t_p1 = np.array(kc_t_p1).mean()

        # Umsetzung der Gleichung 3.1 für die DNI
        return sqrt((((kc-kc_t_m1)**2)+((kc-kc_t_p1)**2))/2)

    def V_combria_GHI(self):
        # Index nach Combria
        # Standardabweichung der zeitlichen Unterschiede von Kc
        """
        Eq. (3.2)
        paper uses DNI values. Here GHI
        can reach values above 1
        Coimbra, C.F.M., J. Kleissl, R. Marquez, 2013: Overview of
        Solar-Forecasting Methods and a Metric for Accuracy Evaluation. – In: Kleissl, J. (Ed.): Solar Energy Forecasting and
        Resource Assessment. Oxford, 171–194.

        Returns
        -------
        V

        """
        V=0

        for i in range(len(self.rad_df['ghi'])-1):
            kc = self.rad_df['ghi'].iloc[i]/self.cs['ghi'].iloc[i]
            kc_t_p1 = self.rad_df['ghi'].iloc[i+1]/self.cs['ghi'].iloc[i+1]
            delta_kc = kc-kc_t_p1
            V = V+(delta_kc)**2

        # Umsetzung der Gleichung 3.2 für die GHI
        return sqrt(V/(i+1))

    def V_combria_DNI(self):
        # Index nach Combria | Tabelle 4
        """
        Eq. (3.2)
        paper uses DNI values
        can reach values above 1
        Coimbra, C.F.M., J. Kleissl, R. Marquez, 2013: Overview of
        Solar-Forecasting Methods and a Metric for Accuracy Evaluation. – In: Kleissl, J. (Ed.): Solar Energy Forecasting and
        Resource Assessment. Oxford, 171–194.

        Returns
        -------
        V

        """
        V=0

        for i in range(len(self.rad_df['dni'])-1):
            kc = self.rad_df['dni'].iloc[i]/self.cs['dni'].iloc[i]
            kc_t_p1 = self.rad_df['dni'].iloc[i+1]/self.cs['dni'].iloc[i+1]
            delta_kc = kc-kc_t_p1
            V = V+(delta_kc)**2

        # Umsetzung der Gleichung 3.2 für die DNI
        return sqrt(V/(i+1))

    def VI_stein_GHI(self):
        # Index nach Stein
        """
        Eq. (3.2)
        paper uses DNI values. Here GHI
        can reach values above 1
        Stein, J.S., M.J. Reno, C. Hansen, 2012: The variability index: a new and novel metric for quantifying irradiance and
        PV output variability. – In: World Renewable Energy Forum,
        Denver, CO.

        Returns
        -------
        VI
        """
        return (sum(((self.rad_df['ghi'].diff().dropna()**2)+1)**(1/2)))/sum(((self.cs['ghi'].diff().dropna()**2)+1)**(1/2))

    def VI_stein_DNI(self):
        # Index nach Stein | Tabelle 4
        """
        Eq. (3.2)
        paper uses DNI values.
        can reach values above 1
        Stein, J.S., M.J. Reno, C. Hansen, 2012: The variability index: a new and novel metric for quantifying irradiance and
        PV output variability. – In: World Renewable Energy Forum,
        Denver, CO.

        Returns
        -------
        VI
        """
        # Umsetzung der Gleichung 3.3 für die DNI
        return (sum(((self.rad_df['dni'].diff().dropna()**2)+1)**(1/2)))/sum(((self.cs['dni'].diff().dropna()**2)+1)**(1/2))

    def Perez_2011_DNI(self):
        # ramp rates nach Perez
        # mean,standard deviation, max
        """
        Source as paper: https://www.sciencedirect.com/science/article/pii/S0038092X11000995

        Returns
        -------
        (delta_DNI_mean, delta_DNI_std, delta_DNI_max, delta_kc_DNI_mean, delta_kc_DNI_std, delta_kc_DNI_max)

        """

        delta_DNI_mean = self.rad_df['dni'].diff().abs().mean()
        delta_DNI_std = self.rad_df['dni'].diff().abs().std()
        delta_DNI_max = self.rad_df['dni'].diff().abs().max()

        delta_kc_DNI_mean = (self.rad_df['dni']/self.cs['dni']).diff().abs().mean()
        delta_kc_DNI_std = (self.rad_df['dni']/self.cs['dni']).diff().abs().std()
        delta_kc_DNI_max = (self.rad_df['dni']/self.cs['dni']).diff().abs().max()

        return delta_DNI_mean, delta_DNI_std, delta_DNI_max, delta_kc_DNI_mean, delta_kc_DNI_std, delta_kc_DNI_max

    def Perez_2011_GHI(self):
        # ramp rates nach Perez
        # mean,standard deviation, max
        """
        Source as paper: https://www.sciencedirect.com/science/article/pii/S0038092X11000995

        Perez, R., S. Kivalov, J. Schlemmer, K. Hemker Jr.,
        T. Hoff, 2011: Parameterization of site-specific short-term
        irradiance variability. – Sol. Energy 85, 1343–1353, DOI:
        10.1016/j.solener.2011.03.016.

        Returns
        -------
        (delta_GHI_mean, delta_GHI_std, delta_GHI_max, delta_kc_GHI_mean, delta_kc_GHI_std, delta_kc_GHI_max)

        """

        delta_GHI_mean = self.rad_df['ghi'].diff().abs().mean()
        delta_GHI_std = self.rad_df['ghi'].diff().abs().std()
        delta_GHI_max = self.rad_df['ghi'].diff().abs().max()

        delta_kc_GHI_mean = (self.rad_df['ghi']/self.cs['ghi']).diff().abs().mean()
        delta_kc_GHI_std = (self.rad_df['ghi']/self.cs['ghi']).diff().abs().std()
        delta_kc_GHI_max = (self.rad_df['ghi']/self.cs['ghi']).diff().abs().max()

        return delta_GHI_mean, delta_GHI_std, delta_GHI_max, delta_kc_GHI_mean, delta_kc_GHI_std, delta_kc_GHI_max

    def Overshoot_fun(self):
        # Ermittelt, wann die Messung 5% bzw. 10% über der Clear-Sky-Vorhersage ist | Tabelle 5
        """
        Returns
        -------
        (counts of five_persent_cs_GHI, counts of ten_percent_cs_GHI)
        """
        five_percent_cs_GHI = self.rad_df['ghi'] > self.cs['ghi']*1.05
        ten_percent_cs_GHI = self.rad_df['ghi'] > self.cs['ghi']*1.1

        # Die Summe der Anzahl an overshoots wird ermittelt
        return sum(five_percent_cs_GHI), sum(ten_percent_cs_GHI)

    def CSFD_DNI(self):
        """
        Kraas B., M. Schroedter-Homscheidt, R. Madlener, 2013:
        Economic Merits of a state-of-the-art concentrating Solar
        Power Forecasting System for Participation in the Spanish Electricity Market. – Sol. Energy 93, 244–255, DOI:
        10.1016/j.solener.2013.04.012.

        Returns
        -------
        CSFD_DNI
        """

        above_fifteen = self.rad_df['dni'].diff().abs()>(self.cs['dni']*0.15)
        CSFD_DNI = ((np.sign(self.rad_df['dni'].diff()).diff().ne(0))*above_fifteen).sum()
        return CSFD_DNI

    def CSFD_GHI(self):
        # Number of Changes in the first Derivation (CSFD). Es werden nur die Extrema verwendet, die mehr als 15% außeinander liegen
        """
        Kraas B., M. Schroedter-Homscheidt, R. Madlener, 2013:
        Economic Merits of a state-of-the-art concentrating Solar
        Power Forecasting System for Participation in the Spanish Electricity Market. – Sol. Energy 93, 244–255, DOI:
        10.1016/j.solener.2013.04.012.

        Returns
        -------
        CSFD_GHI
        """
        # Ermittelt die Summe der Vorzeichenwechsel
        above_fifteen = self.rad_df['ghi'].diff().abs()>(self.cs['ghi']*0.15)
        CSFD_GHI = ((np.sign(self.rad_df['ghi'].diff()).diff().ne(0))*above_fifteen).sum()
        return CSFD_GHI

    def envelope_calc_DNI(self):
        # Berechnung der Umschlagkurven für die Direktstrahlung
        """
        Jung, S., 2015: Variabilität der solaren Einstrahlung in 1-Minuten aufgelösten Strahlungszeitserien. – Master thesis, Universität Augsburg, 2015.

        Returns
        -------
        upper_envelope, lower_envelope | pd.Series
        """
        upper_envelope = [self.rad_df['dni'].iloc[0]]
        lower_envelope = [self.rad_df['dni'].iloc[0]]
        rad_df_intervals = []
        for i in range(len(self.rad_df['dni'])-3):
            rad_df_intervals.append(self.rad_df['dni'].iloc[i:i+4])
        for i in range(len(rad_df_intervals)):

            # rad_df_intervals_temp = rad_df_intervals[i].loc[rad_df_intervals[i]!=rad_df_intervals[i].min()]
            # rad_df_intervals_temp = rad_df_intervals_temp.loc[rad_df_intervals_temp!=rad_df_intervals_temp.max()]

            u_extrema = rad_df_intervals[i].max()
            l_extrema = rad_df_intervals[i].min()
            # u_extrema = rad_df_intervals[i].nlargest(2)[-1]
            # l_extrema = rad_df_intervals[i].nsmallest(2)[-1]

            for j in range(30):
                try:
                    x_m = self.rad_df['dni'][rad_df_intervals[i].index[0]-timedelta(minutes=1+j)]
                except:
                    x_m = rad_df_intervals[i].iloc[0]
                    pass
                try:
                    x_p = self.rad_df['dni'][rad_df_intervals[i].index[-1]+timedelta(minutes=1+j)]
                except:
                    x_p = rad_df_intervals[i].iloc[-1]
                    pass
                if x_m > u_extrema:
                    u_extrema = x_m
                    break
                if x_p > u_extrema:
                    u_extrema = x_p
                    break
                if x_m < l_extrema:
                    l_extrema = x_m
                    break
                if x_p < l_extrema:
                    l_extrema = x_p
                    break
            upper_envelope.append(u_extrema)
            lower_envelope.append(l_extrema)
        upper_envelope.append(u_extrema)
        lower_envelope.append(l_extrema)
        upper_envelope.append(self.rad_df['dni'].iloc[-1])
        lower_envelope.append(self.rad_df['dni'].iloc[-1])
        upper_envelope = pd.DataFrame(data=upper_envelope, index=self.rad_df.index)
        lower_envelope = pd.DataFrame(data=lower_envelope, index=self.rad_df.index)

        return upper_envelope, lower_envelope

    def integral_UML_DNI(self):
        # Integral zwischen der oberen und unteren Umschlagskurve
        upper_envelope, lower_envelope = self.envelope_calc_DNI()
        return (upper_envelope - lower_envelope).mean().values[0]

        # for DNI only. "The integral indices are applied for DNI only, because unlike GHI it has no overshooting effects."

    def integral_UMC_DNI(self):
        # Integral zwischen der oberen Umschlagskurve und der Clear-Sky-Vorhersage
        upper_envelope, lower_envelope = self.envelope_calc_DNI()
        return abs((upper_envelope.squeeze() - self.cs['dni']).mean())
        # for DNI only. "The integral indices are applied for DNI only, because unlike GHI it has no overshooting effects."

    def integral_LMA_DNI(self):
        # Integral zwischen der unteren Umschlagskurve und der X-Achse (Abscissa)
        upper_envelope, lower_envelope = self.envelope_calc_DNI()
        return lower_envelope.mean().values[0]
        # for DNI only. "The integral indices are applied for DNI only, because unlike GHI it has no overshooting effects."

    def envelope_calc_GHI(self):
        # Berechnung der Umschlagskurvenn für die Globalstrahlung
        """
        Jung, S., 2015: Variabilität der solaren Einstrahlung in 1-Minuten aufgelösten Strahlungszeitserien. – Master thesis, Universität Augsburg, 2015.

        Returns
        -------
        upper_envelope, lower_envelope | pd.Series
        """
        upper_envelope = [self.rad_df['ghi'].iloc[0]]
        lower_envelope = [self.rad_df['ghi'].iloc[0]]
        rad_df_intervals = []
        for i in range(len(self.rad_df['ghi'])-3):
            rad_df_intervals.append(self.rad_df['ghi'].iloc[i:i+4])
        for i in range(len(rad_df_intervals)):

            # rad_df_intervals_temp = rad_df_intervals[i].loc[rad_df_intervals[i]!=rad_df_intervals[i].min()]
            # rad_df_intervals_temp = rad_df_intervals_temp.loc[rad_df_intervals_temp!=rad_df_intervals_temp.max()]

            u_extrema = rad_df_intervals[i].max()
            l_extrema = rad_df_intervals[i].min()
            # u_extrema = rad_df_intervals[i].nlargest(2)[-1]
            # l_extrema = rad_df_intervals[i].nsmallest(2)[-1]

            for j in range(30):
                try:
                    x_m = self.rad_df['ghi'][rad_df_intervals[i].index[0]-timedelta(minutes=1+j)]
                except:
                    x_m = rad_df_intervals[i].iloc[0]
                    pass
                try:
                    x_p = self.rad_df['ghi'][rad_df_intervals[i].index[-1]+timedelta(minutes=1+j)]
                except:
                    x_p = rad_df_intervals[i].iloc[-1]
                    pass
                if x_m > u_extrema:
                    u_extrema = x_m
                    break
                if x_p > u_extrema:
                    u_extrema = x_p
                    break
                if x_m < l_extrema:
                    l_extrema = x_m
                    break
                if x_p < l_extrema:
                    l_extrema = x_p
                    break
            upper_envelope.append(u_extrema)
            lower_envelope.append(l_extrema)
        upper_envelope.append(u_extrema)
        lower_envelope.append(l_extrema)
        upper_envelope.append(self.rad_df['ghi'].iloc[-1])
        lower_envelope.append(self.rad_df['ghi'].iloc[-1])
        upper_envelope = pd.DataFrame(data=upper_envelope, index=self.rad_df.index)
        lower_envelope = pd.DataFrame(data=lower_envelope, index=self.rad_df.index)

        return upper_envelope, lower_envelope

    def integral_UML_GHI(self):
        upper_envelope, lower_envelope = self.envelope_calc_GHI()
        return (upper_envelope - lower_envelope).mean().values[0]

        # for DNI only. "The integral indices are applied for DNI only, because unlike GHI it has no overshooting effects."

    def integral_UMC_GHI(self):
        upper_envelope, lower_envelope = self.envelope_calc_GHI()
        return abs((upper_envelope.squeeze() - self.cs['ghi']).mean())
        # absolute Value for normalization for the neural network
        # for DNI only. "The integral indices are applied for DNI only, because unlike GHI it has no overshooting effects."

    def integral_LMA_GHI(self):
        upper_envelope, lower_envelope = self.envelope_calc_GHI()
        return lower_envelope.mean().values[0]
        # for DNI only. "The integral indices are applied for DNI only, because unlike GHI it has no overshooting effects."


    def output_DNI_variables(self):
    # gibt die Variablen für die Kassifikation mithilfe der DNI aus
        return self.sigma_skartveit_DNI(), self.V_combria_DNI(), self.VI_stein_DNI(), *self.Perez_2011_DNI(), self.CSFD_DNI(), self.integral_UML_DNI(), self.integral_UMC_DNI(), self.integral_LMA_DNI()

    def output_GHI_variables(self):
    # gibt die Variablen für die Kassifikation mithilfe der GHI aus
        return self.sigma_skartveit_GHI(), self.V_combria_GHI(), self.VI_stein_GHI(), *self.Perez_2011_GHI(), *self.Overshoot_fun(), self.CSFD_GHI(), self.integral_UML_GHI(), self.integral_UMC_GHI(), self.integral_LMA_GHI()


